In [1]:
from cpmpy import *

In [2]:
from numpy.random import randint
import numpy as np
from numpy.linalg import norm

INFTY = 10000

## Knapsack problem

In [3]:
#knapsack problem
def get_knapsack_problem(N=8, capacity=35):
    np.random.seed(0)
    
    items = boolvar(shape=N, name="items")

    values = randint(0,10,size=N)
    weights = randint(0,10, size=N)

    model = Model(maximize=sum(items * values))
    model += sum(items * weights) <= capacity
    
    return model, (items, values, weights, capacity)

In [4]:
model, (items, values, weights, capacity) = get_knapsack_problem()
assert model.solve()
print("Objective value:",model.objective_value())
print("Used capacity:", sum(items.value() * weights))

print(f"{values = }")
print(f"{weights = }")
print(f"{capacity = }")

print(f"Items = {items.value()}")

Objective value: 32
Used capacity: 31
values = array([5, 0, 3, 3, 7, 9, 3, 5])
weights = array([2, 4, 7, 6, 8, 8, 1, 6])
capacity = 35
Items = [ True False False  True  True  True  True  True]


In [5]:
# User query
# "I want my solution to really contain item 1 and 2"
model += all(items[[1,2]])
assert model.solve()

x_hat = items.value()
print("Objective value:",model.objective_value())
print("Used capacity:", sum(x_hat * weights))

print(f"Items: {x_hat}")

Objective value: 29
Used capacity: 35
Items: [ True  True  True False  True  True False  True]


## Inverse optimize
Apply the cutting plane algorithm for mixed integer inverse optimization.
Here, the goal is to find new cutting planes restricting the inverse feasible space. This space determines the set of coefficients of the objective function that makes the given x_hat solution optimal.

For this we define a master problem (MP) and sub-problem (SP). The master problem iteratively perturbes the objective vector. The SP - which is the orignal forward problem - is then solved to optimality using this new objective function. Lastly, this newly found solution is used to create a new cut in the master problem.

In [7]:
def inverse_optimize(SP, c, x, x_hat, keep_static=None):
    
    # Decision variable for new parameter vector
    d = intvar(0,INFTY, shape=len(x_hat), name="d")

    # create the master problem
    MP = SolverLookup.get("gurobi")
    MP.minimize(norm(c-d,1))
    MP += SP.constraints
    
    if keep_static is not None:
        MP += d[keep_static] == c[keep_static]

    while MP.solve():
        # find new d
        new_d = d.value()
        print(f"New values = {new_d}", end=", \t")
        
        SP.maximize(sum(new_d * x))
        SP.solve()
        print(f"New cut = {x.value()}")

        if sum(new_d * x_hat) >= sum(new_d * x.value()):
            # solution is optimal
            break

        MP += sum(d * x_hat) >= sum(d * x.value())
    return new_d

SP, (x, values, weights,_) = get_knapsack_problem()

print("Original values:", values)
keep_static = [0,3,4,5,6,7]
inverse_optimize(SP, values, x, x_hat, keep_static)

Original values: [5 0 3 3 7 9 3 5]
New values = [5 0 3 3 7 9 3 5], 	New cut = [ True False False  True  True  True  True  True]
New values = [5 0 6 3 7 9 3 5], 	New cut = [ True False  True False  True  True  True  True]
New values = [5 3 3 3 7 9 3 5], 	New cut = [ True  True False  True  True  True  True  True]
New values = [5 3 6 3 7 9 3 5], 	New cut = [ True  True False  True  True  True  True  True]


array([5, 3, 6, 3, 7, 9, 3, 5])